In [ ]:
from PIL import Image
import numpy as np
import sys
from os import listdir
from os.path import isfile, join
from random import sample

np.set_printoptions(threshold=sys.maxsize)

#colors
BLACK = (0, 0, 0)
WHITE = (255, 255, 255)

BLANK_PIXEL_VALUE = -9999

TILE_SIZE = 1000 # in pixels
DATA_SOURCE_FOLDER = "VVD/5"
DATA_TARGET_FOLDER = "VVD_1000"
TEST_FOLDER = join(DATA_TARGET_FOLDER, "test")
TRAIN_FOLDER = join(DATA_TARGET_FOLDER, "train")

TRAIN_RATIO = 0.8
TEST_RATIO = 1-TRAIN_RATIO

files = []
for file_name in listdir(DATA_SOURCE_FOLDER):
    file_path = join(DATA_SOURCE_FOLDER, file_name)
    if isfile(file_path):
        files.append(file_path)

main_idx = 0
for file_path in files:
    file_name = file_path.split("/")[2]
    
    image = Image.open(file_path)
    image_array = np.array(image)   # "data" is a height x width x 4 numpy array

    image_size = image_array.shape[0]
    
    split_size = image_size / TILE_SIZE
    number_of_tiles = split_size ** 2
    test_tile_indices = sample(range(0, 100), int(number_of_tiles*TEST_RATIO))

    print(f"{main_idx}. {file_name}")
    file_name_base = file_name.split(".")[0]

    if (split_size).is_integer():
        split_size = int(split_size)
        
        idx = 0    
        for tile_index_i in range(split_size):
            for tile_index_j in range(split_size):

                target_folder = TRAIN_FOLDER
                if idx in test_tile_indices:
                    target_folder = TEST_FOLDER           
                        
                row_from = tile_index_i*TILE_SIZE
                row_to = tile_index_i*TILE_SIZE + TILE_SIZE
                col_from = tile_index_j*TILE_SIZE
                col_to = tile_index_j*TILE_SIZE + TILE_SIZE

                tile_array = image_array[row_from:row_to,col_from:col_to]
                
                blank_elements = np.count_nonzero(tile_array == BLANK_PIXEL_VALUE)
                if blank_elements <= number_of_tiles/2:
                    tile_file_name = f"{file_name_base}_{tile_index_i}_{tile_index_j}.tif"
                    tile_file_path = join(target_folder, tile_file_name)

                    tile = Image.fromarray(tile_array)
                    tile.save(tile_file_path)
                    #tile.show()
                    
                idx+=1
    main_idx+=1